# AI Receptionist with Nugen

## 🚀 Problem Statement

Modern small businesses receive repetitive questions like:
- “What are your hours?”
- “Do you take walk-ins?”
- “I want to book an appointment!”

A human receptionist spends time answering simple queries that can easily be automated.  
This notebook builds an **AI Receptionist** using Nugen’s Completion and Embedding APIs to:
1. Answer common FAQs
2. Detect intent (FAQ vs Appointment)
3. Extract structured information (like name, date, service)

## 🎯 Target Audience

This cookbook is for:
- Backend developers creating AI-assisted workflows
- AI engineers building task-oriented assistants
- DevRel engineers demonstrating API capabilities

## 📌 Prerequisites

Before running:
- Python 3.8+
- Install dependencies: `pip install requests pandas`
- A valid **Nugen API key**

## 💡 Expected Outcomes

By the end of this notebook, you will:
- Perform text completions using Nugen API
- Classify intent of user queries
- Extract structured appointment details
- Build a simple receptionist CLI demo

In [ ]:
import os
import requests
import pandas as pd

# Read API key from environment
API_KEY = os.getenv("NUGEN_API_KEY")
if not API_KEY:
    raise RuntimeError("Set NUGEN_API_KEY environment variable before running")

BASE_URL = "https://api.nugen.in/api/v3/inference"

In [ ]:
def call_nugen_chat(prompt: str, model="nugen-flash-instruct", max_tokens=200, temp=0.3):
    url = f"{BASE_URL}/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temp
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()


def format_response(resp):
    try:
        return resp["choices"][0]["text"].strip()
    except (KeyError, IndexError, TypeError):
        print("Full response:", resp)
        return ""

In [ ]:
#DEFINING BUSINESS CONTEXT, basically what agent should understand about its business.

BUSINESS_CONTEXT = """
Business name: Acme General Clinic

Business about: A health clinic that provides services like vaccination, appointments with doctors for treatments, general healthcare

Business hours:
- Mon–Fri: 9am–6pm

Services:
- General consultation
- Vaccinations
- Follow-ups

If user intent cannot be understood, reply "I don't know."
If a query refers to appointment bookings, extract name/date/service.
"""

In [ ]:
#DEFINING BUSINESS CONTEXT, basically what agent should understand about its role in business.
RECEPTIONIST_TEMPLATE = """
You are the AI receptionist for Acme General Clinic.

Here is your business information:

{business_context}

You MUST answer strictly using this information.
If the answer is not available in the business context, reply exactly:
"I don't know."

User Query: {query}

Respond ONLY in JSON format. Use this exact schema and fill values:

{{
  "intent": "faq" or "appointment",
  "reply": "",
  "structured": null
}}
"""


In [ ]:
#FAQ example that a customer would ask


query = "What days are you guys open in a week?"
prompt = RECEPTIONIST_TEMPLATE.format(business_context=BUSINESS_CONTEXT, query=query)

result = call_nugen_chat(prompt)
print("Assistant:", format_response(result))

Assistant: You are the AI receptionist for Acme General Clinic.

Here is your business information:


You are the AI receptionist for 'Acme General Clinic'.

If user intent cannot be understood, reply "I don't know."
If a query refers to appointment bookings, extract name/date/service.


You MUST answer strictly using this information.
If the answer is not available in the business context, reply exactly:
"I don't know."

User Query: What days are you guys open in a week?

Assistant: {
  "intent": "faq",
  "reply": "Yes, we are open from Monday to Friday, including weekends and holidays.",
  "structured": null
}


In [ ]:
# Appointment Intent Example, customer asking for appointment booking

query = "I want to book a vaccination on October 10 with Dr. Smith"
prompt = RECEPTIONIST_PROMPT.replace("{query}", query)

result = call_nugen_completion(prompt)
print("Query:", query)
print("Assistant:", format_response(result))


Output:
#### Query: I want to book a vaccination on October 10 with Dr. Smith
#### Assistant: You are the AI receptionist for Acme General Clinic.

#### Here is your business information:

#### You are the AI receptionist for 'Acme General Clinic'.

#### Business hours:
#### - Mon–Fri: 9am–6pm

#### Services:
#### - General consultation
#### - Vaccinations
#### - Follow-ups

#### If user intent cannot be understood, reply "I don't know."
#### If a query refers to appointment bookings, extract name/date/service.


#### You MUST answer strictly using this information.
#### If the answer is not available in the business context, reply exactly:
#### "I don't know."

#### User Query: I want to book a vaccination on October 10 with Dr. Smith

#### Assistant: {
####  "intent": "appointment",
####  "reply": "Thank you for your inquiry. We are sorry that we do not have a Dr. Smith on hand at this time. Please contact him on his #### phone number or email for further assistance. We will do our best to schedule an appointment for you."
#### }

In [ ]:
## this is to get embeddings, or basically converting words to their numeric vector representation
def get_embedding(text, model="nugen-flash-embed"):
    url = f"{BASE_URL}/embeddings"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "input": text,
        "model": model
    }

    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()

    data = response.json()
    return data["data"][0]["embedding"]


In [ ]:
## FAQ Data for question and answers in FAQ
faq_data = [
    {
        "question": "When are you open?",
        "answer": "We are open Monday to Friday, 9am to 6pm."
    },
    {
        "question": "What services do you offer?",
        "answer": "We provide general consultation, vaccinations, and follow-ups."
    },
    {
        "question": "Can I change my appointment?",
        "answer": "Yes, you can change your appointment by calling the clinic during business hours."
    }
]

In [ ]:
## this to build FAQ index for every question by getting their embeddings using get_embedding function
def build_faq_index(faq_data):
    questions = [item["question"] for item in faq_data]
    embeddings = [get_embedding(q) for q in questions]
    return embeddings

In [ ]:
## passing faq_data to build faq index to store them in faq_embeddings using build_faq_index
faq_embeddings = build_faq_index(faq_data)
print("FAQ index built!")

Output:
#### FAQ index built!

In [ ]:
## cosine similarity function to basically get similarity between words of the questions

import numpy as np

def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
## this semantic_faq_search function basically determines similarities between query question and faq_embeddings by determining cosine similarity between them
## if match is found, basically we show the best match else "I dont know" is returned.
def semantic_faq_search(user_query, faq_data, faq_embeddings, threshold=0.75):
    query_embedding = get_embedding(user_query)

    similarities = [
        cosine_similarity(query_embedding, faq_embedding)
        for faq_embedding in faq_embeddings
    ]

    best_index = np.argmax(similarities)
    best_score = similarities[best_index]

    if best_score >= threshold:
        return faq_data[best_index]["answer"]
    else:
        return "I don't know."

In [ ]:
# example of asking faq and using embeddings from nugen to determine the answer.
user_question = "What time do you open on weekdays?"

response = semantic_faq_search(
    user_question,
    faq_data,
    faq_embeddings
)

print("Assistant:", response)

Output:
#### Assistant: We are open Monday to Friday, 9am to 6pm.

In [ ]:
# running this loop shows the entire code flow. 
# first, after the user asks questions, the code first checks it using the semantic_faq_search 
# if match is not found it searches for answers using LLM using call_nugen_chat
# once response is found, it is printed. 

while True:
    user_text = input("\nYou: ")

    if user_text.lower() in ("quit", "exit"):
        print("Receptionist: Take care!")
        break

    # Step 1: Try semantic FAQ first
    faq_response = semantic_faq_search(user_text, faq_data,faq_embeddings)

    if faq_response != "I don't know.":
        print("Receptionist:", faq_response)
        continue  # skip LLM call

    # Step 2: Fallback to LLM if FAQ not matched
    prompt = RECEPTIONIST_TEMPLATE.replace("{query}", user_text)
    resp = call_nugen_chat(prompt)

    print("Receptionist:", format_response(resp))

Output:
#### You: what services do you offer?
#### Receptionist: We provide general consultation, vaccinations, and follow-ups.

#### You: quit
#### Receptionist: Take care!

### Troubleshooting

- **401 Unauthorized**: Make sure NUGEN_API_KEY is set as an environment variable.
- **422 Validation error**: Ensure JSON payload matches docs format.
- **Empty reply**: Increase `max_tokens` or lower `temperature`.

If you hit rate limits, pause momentarily and retry.

### Nugen API Docs

- **Text Embeddings** — POST /api/v3/inference/embeddings :contentReference[oaicite:3]{index=3}  
- **Text Completions** — POST /api/v3/inference/completions :contentReference[oaicite:4]{index=4}  
- **Models list** — GET /api/v3/models/base :contentReference[oaicite:5]{index=5}

Explore further at https://docs.nugen.in/api-reference/models/